# Data prep

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.learn.python.learn.estimators.estimator import SKCompat
from tensorflow.contrib import rnn
from tensorflow.python.ops import array_ops as array_ops_
learn = tf.contrib.learn

In [2]:
pd_ads=pd.read_csv('D:\\sale\\t_ads.csv')
pd_comment=pd.read_csv('D:\\sale\\t_comment.csv')
pd_order=pd.read_csv('D:\\sale\\t_order.csv')
pd_product=pd.read_csv('D:\\sale\\t_product.csv')
pd_sale=pd.read_csv('D:\\sale\\t_sales_sum.csv')

C:\Users\sjt\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sale_1036=pd_sale[pd_sale['shop_id']==1036]
comment_1036=pd_comment[pd_comment['shop_id']==1036]
order_1036=pd_order[pd_order['shop_id']==1036]
product_1036=pd_product[pd_product['shop_id']==1036]
ads_1036=pd_ads[pd_ads['shop_id']==1036]

sale_1036['dt']=pd.to_datetime(sale_1036['dt'])
comment_1036['create_dt']=pd.to_datetime(comment_1036['create_dt'])
order_1036['ord_dt']=pd.to_datetime(order_1036['ord_dt'])
product_1036['on_dt']=pd.to_datetime(product_1036['on_dt'])
ads_1036['create_dt']=pd.to_datetime(ads_1036['create_dt'])

sale_1036=sale_1036.sort_values(['dt'],ascending=True)
comment_1036=comment_1036.sort_values(['create_dt'],ascending=True)
order_1036=order_1036.sort_values(['ord_dt'],ascending=True)
product_1036=product_1036.sort_values(['on_dt'],ascending=True)
ads_1036=ads_1036.sort_values(['create_dt'],ascending=True)

sale_1036.to_csv('D:\\sale\\sale_1036.csv')
comment_1036.to_csv('D:\\sale\\comment_1036.csv')
order_1036.to_csv('D:\\sale\\order_1036.csv')
product_1036.to_csv('D:\\sale\\product_1036.csv')
ads_1036.to_csv('D:\\sale\\ads_1036.csv')

C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sjt\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html

In [4]:
shop_id=pd_sale['shop_id']
shop_id=list(set(shop_id))
sale_list=[]
comment_list=[]
order_list=[]
product_list=[]
ads_list=[]
sale_list=[]

for i in shop_id:
    sale_list.append(pd_sale[pd_sale['shop_id']==i])
    comment_list.append(pd_comment[pd_comment['shop_id']==i])
    order_list.append(pd_order[pd_order['shop_id']==i])
    product_list.append(pd_product[pd_product['shop_id']==i])
    ads_list.append(pd_ads[pd_ads['shop_id']==i])
    
    sale_list[i-1]=sale_list[i-1].sort_values(['dt'],ascending=True)
    comment_list[i-1]=comment_list[i-1].sort_values(['create_dt'],ascending=True)
    order_list[i-1]=order_list[i-1].sort_values(['ord_dt'],ascending=True)
    product_list[i-1]=product_list[i-1].sort_values(['on_dt'],ascending=True)
    ads_list[i-1]=ads_list[i-1].sort_values(['create_dt'],ascending=True)
    
    sale_list[i-1]['dt']=pd.to_datetime(sale_list[i-1]['dt'])
    comment_list[i-1]['create_dt']=pd.to_datetime(comment_list[i-1]['create_dt'])
    order_list[i-1]['ord_dt']=pd.to_datetime(order_list[i-1]['ord_dt'])
    product_list[i-1]['on_dt']=pd.to_datetime(product_list[i-1]['on_dt'])
    ads_list[i-1]['create_dt']=pd.to_datetime(ads_list[i-1]['create_dt'])
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


In [5]:
for i in range(len(order_list)):
    order_list[i].drop('pid',axis=1,inplace=True)
    order_list[i].drop('shop_id',axis=1,inplace=True)
    order_list[i]=order_list[i].groupby(['ord_dt']).sum()
    
for i in range(len(comment_list)):
    comment_list[i].set_index("create_dt", inplace=True)
    comment_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(ads_list)):
    ads_list[i].set_index("create_dt",inplace=True)
    ads_list[i].drop('shop_id',axis=1,inplace=True)
for i in range(len(sale_list)):
    sale_list[i].set_index("dt",inplace=True)
    sale_list[i].drop('shop_id',axis=1,inplace=True)

In [6]:
date_list=[]
for i in shop_id:
    tmp=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
    date_list.append(pd.concat([tmp,comment_list[i-1], order_list[i-1],ads_list[i-1]], axis=1, join_axes=[tmp.index]))
    date_list[i-1].fillna(value=0,inplace=True)

In [7]:
sale_list

[            sale_amt_3m
 dt                     
 2016-06-30     87360.95
 2016-07-31    196326.00
 2016-08-31    335835.70
 2016-09-30    404198.61
 2016-10-31    382664.23
 2016-11-30    306535.85
 2016-12-31    248421.73
 2017-01-31    219097.84,             sale_amt_3m
 dt                     
 2016-06-30     47995.50
 2016-07-31     99079.11
 2016-08-31    141808.25
 2016-09-30    144865.94
 2016-10-31     90416.49
 2016-11-30     59019.46
 2016-12-31     55028.44
 2017-01-31     79214.06,             sale_amt_3m
 dt                     
 2016-06-30    100554.27
 2016-07-31    122139.45
 2016-08-31    197950.82
 2016-09-30    272725.95
 2016-10-31    290275.52
 2016-11-30    223530.49
 2016-12-31    150484.42
 2017-01-31    134667.29,             sale_amt_3m
 dt                     
 2016-06-30    166206.97
 2016-07-31    137186.21
 2016-08-31    101048.68
 2016-09-30     61281.42
 2016-10-31     25298.68
 2016-11-30     22308.22
 2016-12-31     27053.83
 2017-01-31     47633.91,

In [8]:
order_1036.drop('pid',axis=1,inplace=True)
order_1036=order_1036.groupby(['ord_dt']).sum()

comment_1036.drop('shop_id',axis=1,inplace=True)
order_1036.drop('shop_id',axis=1,inplace=True)
ads_1036.drop('shop_id',axis=1,inplace=True)
sale_1036.drop('shop_id',axis=1,inplace=True)


#order_1036.set_index("ord_dt", inplace=True) order_1036已将ord_dt作为索引
sale_1036.set_index("dt", inplace=True)
comment_1036.set_index("create_dt", inplace=True)
product_1036.set_index("on_dt", inplace=True)
ads_1036.set_index("create_dt", inplace=True)

sale_1036

,sale_amt_3m
dt,
2016-06-30,435382.02
2016-07-31,425521.09
2016-08-31,546715.20
2016-09-30,543089.88
2016-10-31,372576.44
2016-11-30,120428.26
2016-12-31,140046.79
2017-01-31,251676.49


In [9]:
#date_list=pd.date_range('2016-6-1','2017-4-30')
date=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
data=pd.concat([date,comment_1036, order_1036,ads_1036], axis=1, join_axes=[date.index])
data.fillna(value=0,inplace=True)
data.to_csv('D:\\sale\\data_1036.csv')
data

,bad_num,cmmt_num,dis_num,good_num,mid_num,sale_amt,offer_amt,offer_cnt,rtn_cnt,rtn_amt,ord_cnt,user_cnt,charge,consume
2016-08-01,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
2016-08-02,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00
2016-08-03,0.0,0.0,0.0,0.0,0.0,6462.10,2369.94,81.0,2.0,58.12,124.0,124.0,0.00,0.00
2016-08-04,0.0,0.0,0.0,0.0,0.0,5605.59,2290.11,74.0,0.0,0.00,107.0,107.0,0.00,0.00
2016-08-05,0.0,144.0,0.0,144.0,0.0,5328.07,2242.29,72.0,0.0,0.00,104.0,103.0,0.00,0.00
2016-08-06,0.0,198.0,2.0,196.0,2.0,5957.45,2372.53,77.0,0.0,0.00,118.0,118.0,0.00,0.00
2016-08-07,0.0,240.0,0.0,240.0,0.0,5793.08,2289.76,76.0,0.0,0.00,112.0,112.0,0.00,0.00
2016-08-08,0.0,266.0,0.0,266.0,0.0,5626.25,2320.65,75.0,0.0,0.00,109.0,109.0,0.00,0.00
2016-08-09,0.0,360.0,0.0,360.0,0.0,5372.68,2218.75,72.0,0.0,0.00,104.0,104.0,0.00,0.00
2016-08-10,0.0,304.0,0.0,304.0,0.0,5462.70,2204.41,71.0,0.0,0.00,102.0,102.0,0.00,0.00


# RNN

设置参数

In [10]:
HIDDEN_SIZE = 30
NUM_LAYERS = 2

TIMESTEPS = 30
TRAINING_STEPS = 3000
BATCH_SIZE = 32

TRAINING_EXAMPLES = 10000
TESTING_EXAMPLES = 1000
SAMPLE_GAP = 0.01

timesteps=30
num_input=14
num_hidden=30
learning_rate= 0.001
training_steps=10000

 生成训练数据

In [11]:
def generate_data(data,labels):
    X = []
    y = []
    seq=[0,31,61,92,122]

    for j in range(len(sale_list)):
        for i in range(5):
            X.append(np.array(data[j][seq[i]: seq[i] + 30],dtype=np.float32))
            y.append(np.array(labels[j].iloc[i + 2],dtype=np.float32))
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [12]:
train_X,train_y=generate_data(date_list,sale_list)

In [13]:


# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None,1])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, 1]))
}
biases = {
    'out': tf.Variable(tf.random_normal([1]))
}

In [14]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    print(x.shape)
    x = tf.unstack(x, timesteps, 1)
    print(len(x))
    print(x[0].shape)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [ ]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

(?, 30, 14)
30
(?, 14)


In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        #batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: train_X, Y: train_y})
        if True:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_X,
                                                                 Y: train_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    #test_len = 128
    #test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    #test_label = mnist.test.labels[:test_len]
    #print("Testing Accuracy:", \
    #    sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 2, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 3, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 4, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 5, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 6, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 7, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 8, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 9, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 10, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 11, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 12, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 13, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 14, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 15, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 16, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 17, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 18, Minibatch Loss

Step 142, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 143, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 144, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 145, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 146, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 147, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 148, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 149, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 150, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 151, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 152, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 153, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 154, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 155, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 156, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 157, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 158, Minibatch Loss= 0.0000, Training Accuracy= 1.0

定义lstm模型

In [8]:
#弃用
def lstm_model(X, y):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(HIDDEN_SIZE, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * NUM_LAYERS)
    
    output, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    output = tf.reshape(output, [-1, HIDDEN_SIZE])
    
    # 通过无激活函数的全联接层计算线性回归，并将数据压缩成一维数组的结构。
    predictions = tf.contrib.layers.fully_connected(output, 1, None)
    
    # 将predictions和labels调整统一的shape
    labels = tf.reshape(y, [-1])
    predictions=tf.reshape(predictions, [-1])
    
    loss = tf.losses.mean_squared_error(predictions, labels)
    
    train_op = tf.contrib.layers.optimize_loss(
        loss, tf.contrib.framework.get_global_step(),
        optimizer="Adagrad", learning_rate=0.1)

    return predictions, loss, train_op

训练

In [11]:
#弃用
regressor = SKCompat(learn.Estimator(model_fn=lstm_model))

regressor.fit(X, y,  steps=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000263E31B36D8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\sjt\\AppData\\Local\\Temp\\tmpzlj6mtf9'}


ValueError: Dimensions must be equal, but are 60 and 44 for 'rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [?,60], [44,120].